# Linear Optimal Power Flow (LOPF) 
_with the IEEE 24-Bus Reliability Test System (RTS) & PyPSA_

> The IEEE RTS 24-Bus Test system System (original from 1979) is given here: https://icseg.iti.illinois.edu/ieee-24-bus-system/. It is also implemented in [pandapower](https://raw.githubusercontent.com/e2nIEE/pandapower/master/pandapower/networks/power_system_test_case_jsons/case24_ieee_rts.json).

### [Input LOPF vs Non-Linear OPF](https://pypsa.readthedocs.io/en/latest/user-guide/power-flow.html#id4)
|LOPF| Non-Linear OPF|
|----|----|
|- n.buses.{v_nom}| - n.buses.{v_nom, v_mag_pu_set}
|- n.loads.{p_set}| - n.loads.{p_set, q_set}|
|- n.generators.{p_set}| - n.generators.{control, p_set, q_set}|
|- n.storage_units.{p_set}| - n.storage_units.{control, p_set, q_set}|
|- n.stores.{p_set}| - n.stores.{p_set, q_set}|
|- n.shunt_impedances.{g}| - n.shunt_impedances.{b, g}|
|- n.lines.{x}| - n.lines.{x, r, b, g}|
|- n.transformers.{x}| - n.transformers.{x, r, b, g}|
|- n.links.{p_set}| - n.links.{p_set}||

In [ ]:
import cartopy.crs as ccrs
import json
import matplotlib.pyplot as plt
import pandas as pd
import pypsa
import requests

Get the IEEE 24 bus system from pandapower (here old version from 1979):

In [ ]:

url = 'https://raw.githubusercontent.com/e2nIEE/pandapower/master/pandapower/networks/power_system_test_case_jsons/case24_ieee_rts.json'

response = requests.get(url)
response_obj = response.json()["_object"]
response_obj.keys()


In [ ]:
def extract_pandapower_data(response_obj: dict)-> dict[str, pd.DataFrame]:  
    def create_df(component: str) -> pd.DataFrame:
        try:
            data = json.loads(response_obj[component]['_object'])
            data_df = pd.DataFrame(
                data=data['data'], 
                columns=data['columns'], 
                index=data['index']
            ).astype(response_obj[component]['dtype'])
        except (KeyError, TypeError) :
            print(f"ommiting {component=} ...")
            return None
        return data_df

    return {
        component: create_df(component) 
        for component in response_obj.keys()
        if not component.startswith('res_')}

data = extract_pandapower_data(response_obj)
data.keys()

Compose the PyPSA network:

In [ ]:
network = pypsa.Network()
network.set_snapshots(range(24)) #h

network.add(
    "Carrier",
    'conventional',
    nice_name="Conventional",
    color="grey",
)

for index, row in data['bus'].iterrows():
    network.add(
        'Bus', 
        name=f"bus_{index}", 
        v_nom=row['vn_kv'],
        x=data['bus_geodata'].loc[index, 'x'],
        y=data['bus_geodata'].loc[index, 'y'],
        carrier='AC'
    )

gen_poly_cost = data["poly_cost"][(data['poly_cost']['et']=='gen')].set_index('element')

for index, row in data['gen'].iterrows():
    network.add(
        'Generator', 
        name=f"gen_{index}", 
        bus=f"bus_{row['bus']}",
        carrier="conventional",
        p_nom=row['max_p_mw'],
        marginal_cost=gen_poly_cost.loc[index]['cp1_eur_per_mw'],
        marginal_cost_quadratic=gen_poly_cost.loc[index]['cp2_eur_per_mw2'],
    )

# add trafos from pandapower to transformer_types
for _, trafo in data['trafo'].iterrows():
    trafo_name = f"{int(trafo['sn_mva'])} MVA {int(trafo['vn_hv_kv'])}/{int(trafo['vn_lv_kv'])} kV"
    new_trafo = pd.Series(index=network.transformer_types.columns,)
    new_trafo[:'phase_shift'] = [60, *trafo['sn_mva': 'shift_degree']]
    new_trafo['tap_step']=trafo['tap_step_percent']
    new_trafo['tap_side']=0
    new_trafo['tap_neutral']=0
   
    network.transformer_types.loc[trafo_name] = new_trafo

for index, row in data['trafo'].iterrows():
    network.add(
        'Transformer',
        name=f'trafo_{index}',
        bus0=f'bus_{row["hv_bus"]}',
        bus1=f'bus_{row["lv_bus"]}',
        type=f"{int(row['sn_mva'])} MVA {int(row['vn_hv_kv'])}/{int(row['vn_lv_kv'])} kV"
    )

for index, row in data['line'].iterrows():
    network.add(
        'Line',
        name=f'cable_{index}',
        bus0=f'bus_{row["from_bus"]}',
        bus1=f'bus_{row["to_bus"]}',
        length=row['length_km'],
        x= row['x_ohm_per_km']*row['length_km'],
        r=row['r_ohm_per_km']*row['length_km'],
        #b = row['c_nf_per_km']*2*np.pi*60/1000000000,
        s_nom=500
    )

network.plot(
    title='IEEE 24-Bus Reliability Test System (RTS)', 
    transformer_colors='orange',
)

for index, bus in network.buses.iterrows():
    plt.text(bus.x, bus.y, f'{bus.name}', fontsize=8, zorder=99)
plt.tight_layout()

network.generators.T

Total system demand and the demand per node:

In [ ]:
# Add load
total_system_demand_df = pd.DataFrame(
    [
        1775.835, 1669.815, 1590.3, 1563.795, 1563.795, 1590.3, 1961.37, 2279.43,
        2517.975, 2544.48, 2544.48, 2517.975, 2517.975, 2517.975, 2464.965, 2464.965,
        2623.995, 2650.5, 2650.5, 2544.48, 2411.955, 2199.915, 1934.865, 1669.815
    ],
    index=range(1,24+1),
    columns=['System Demand [MW]'],
)

node_demand_share = {
    'bus_0': 0.038,
    'bus_1': 0.034,
    'bus_2': 0.063,
    'bus_3': 0.026,
    'bus_4': 0.025,
    'bus_5': 0.048,
    'bus_6': 0.044,
    'bus_7': 0.06,
    'bus_8': 0.061,
    'bus_9': 0.068,
    'bus_12': 0.093,
    'bus_13': 0.068,
    'bus_14': 0.111,
    'bus_15': 0.035,
    'bus_17': 0.117,
    'bus_18': 0.064,
    'bus_19': 0.045,
}

# scaling factor needed to adjust load to total nominal power in the system
scaling_factor = network.generators.p_nom.sum() / total_system_demand_df['System Demand [MW]'].max()

for index, bus in network.buses.iterrows():
    network.add(
        'Load',
        name=f'demand_{index}',
        bus=f'{bus.name}',
        p_set=list(total_system_demand_df['System Demand [MW]'] * scaling_factor * node_demand_share.get(bus.name,0))
    )
    
network.loads_t.p_set.sum(axis=1).plot(ylim=[0, 1500])


In [ ]:
load_share_2 = network.loads.assign(l=network.loads_t.p_set.mean()).groupby(["bus"]).l.sum()

network.plot(
    bus_sizes=load_share_2 /1000,
    title="demand"
)

In [ ]:
network.lpf()

In [ ]:
p_nom_share = network.generators.assign(g=network.generators.p_nom).groupby(["bus"]).g.sum()

network.plot(
    bus_sizes=p_nom_share /10000,
    title="p_nom",
    flow='mean',
    line_widths=0.5,
)

In [ ]:
network.optimize(solver_name='highs')

In [ ]:
network.generators_t.p

In [ ]:
gen_share = network.generators.assign(g=network.generators_t.p.mean()).groupby(["bus"]).g.sum()

s = network.generators_t.p.loc[0].groupby([network.generators.bus, network.generators.carrier]).sum()

network.plot(
    bus_sizes=s / 1000,
    flow='mean',
    line_widths = 0.1,
    link_widths=0.1,
)

## Resources

- https://pandapower.readthedocs.io/en/v2.1.0/elements/gen.html


updated IEEE 24 bus RTS (2017):
- https://backend.orbit.dtu.dk/ws/portalfiles/portal/120568114/
An_Updated_Version_of_the_IEEE_RTS_24Bus_System_for_Electricty_Market_an....pdf
- https://github.com/chrord/Energy_and_reserves_dispatch_with_DRJCC/blob/master/Codes/src/Data_generation/RTS_Data2.m

- https://www.gams.com/45/psoptlib_ml/libhtml/psoptlib_MultiperiodDCOPF24bus.html